#Sourcing and Preparing the Data From Central Car Auction

**Data Source**
Webpage:  http://www.centralcarauctions.com/trade/vehicles/price-guide/price-guide?page=1

Date Accessed: July 2014 

Steps in this notebook: 

1. Read in downloaded copy of webpage and find the car info with BS4.  
2. Extract data to a structured pandas data frame.
3. Data pre-processing/filtering/munging/data type convertion.
4. Feature Encoding of categorical variables.
5. Write raw and encoded data to csv.

### Load HTML and Find List Items

In [1]:
%pylab --no-import-all inline
from bs4 import BeautifulSoup
import pandas as pd 

soup = BeautifulSoup(open('centralcarauctions.html'))
entries = soup.find_all('li', class_="p-g-item") 

Populating the interactive namespace from numpy and matplotlib


### Extract Data to Data Frame

In [2]:
from collections import defaultdict

data_dict = defaultdict(list) 

for ent in entries:
    items = ent.find_all('span')
    data_dict['make'].append(items[0].string)
    
    text = items[1].string.split(' ')
    data_dict['model'].append(text[0])
    data_dict['trim'].append(' '.join(text[1:]))
    
    data_dict['class'].append(items[2].string)
    data_dict['year'].append(items[4].string)
    data_dict['MOT'].append(items[6].string)
    data_dict['mileage'].append(items[8].string)
    data_dict['price'].append(items[9].string)

data_headers = ['make', 'model', 'trim', 'class', 'year', 'mileage', 'MOT', 'price']
df_raw = pd.DataFrame(data_dict, columns=data_headers)
df_raw.head()

,make,model,trim,class,year,mileage,MOT,price
0,ALFA ROMEO,147,COLLEZIONE TS - 1598CC,5dr Hatchback,Jun 2008 (08),"58,666",None,"£1,850.00"
1,ALFA ROMEO,147,Q2 JTDM 16V - 1910CC,3dr Hatchback,Sep 2007 (57),"69,086",Sep 2014,"£2,625.00"
2,ALFA ROMEO,156,T SPARK LUSSO - 1747CC,5dr Estate,May 2003 (03),"85,959",Jan 2015,£325.00
3,ALFA ROMEO,156,VELOCE JTS - 1970CC,4dr Saloon,Sep 2004 (54),"98,670",Sep 2014,£300.00
4,ALFA ROMEO,159,LUSSO JTDM S-WAGON - 1910CC,5dr Estate,Dec 2007 (57),"87,890",May 2015,"£3,500.00"


### Write raw data.csv

In [3]:
df_raw.to_csv('car_auction_raw.csv', index=False)

# Variable Descriptions

* **make**:    Categorical 
* **model**:   Categorical
* **trim**:    Categorical but high number of choices
* **desc**:    Categorical 
* **year**:    Ordinal
* **mileage**: Number, Integer
* **MOT**:     Ordinal (Based on amount of time remaining)
* **price**:   Number, Float

##Encoding Strategy 

* Categorical: Numbered according to Alphabetised unique entries.  
* Ordinal Variables: Increasing or decreasing. 

## First Guess at Informative Feature Subset
1. **Make** or **Model** depending on granularity needed, probabliy dont need both.  
2. **desc** good at representing general class of the vehical  
3. **year**:   Likely to correlate negatively with price
4. **mileage**: Similar as above 
5. **MOT**:     Based on amount of time remaining, higher may correlate with higher price  

Target Variable: **price**   


# Data Pre-processing  

In [4]:
df_pro = df_raw.copy()

### Dealing with Dates 

In [5]:
# Convert MOT and Year to date time 
df_pro.MOT = df_raw.MOT
df_pro.MOT = pd.to_datetime(df_raw.MOT, format='%b %Y', coerce=True)
df_pro.year = df_raw.year
df_pro.year = df_raw.year.str.replace(pat=' \(.+\)', repl='')
df_pro.year = pd.to_datetime(df_pro.year, format='%b %Y')

### Convert Price and Mileage to Floats

In [6]:
# Strip commas and missing values
df_pro.mileage = df_pro.mileage.str.replace(',', '')
df_pro.mileage = df_pro.mileage.str.replace('not showing', 'NAN')
df_pro.mileage = df_pro.mileage.str.replace('Not Showing', 'NAN')
df_pro.mileage = df_pro.mileage.str.replace('.', 'NAN')

# Convert km to miles
km_idx = df_pro.mileage.str.contains('km')
for idx in km_idx.nonzero()[0]:
    df_pro.loc[idx, 'mileage'] = 5 * int(df_pro.loc[idx, 'mileage'].split(' ')[0]) / 8. 
    
df_pro.mileage = df_pro.mileage.astype(float)

In [7]:
# Convert Price To string then munge then to floats
df_pro.price = df_pro.price.str.replace(',', '')
df_pro.price = df_pro.price.str.replace(u'\xa3', '')
df_pro.price = df_pro.price.astype(float)

### Save Preprocessed Data 

In [8]:
df_pro.to_csv('car_auction_processed.csv', index=False)

# Feature Encoding

In [9]:
df_enc = df_pro.copy()

In [10]:
# Ecode Dates as ordinals  
MOT_decoder = np.unique(df_enc.MOT)
df_enc['MOT_ord'] = np.unique(df_enc.MOT, return_inverse=True)[1]

year_decoder = np.unique(df_enc.year)
df_enc['year_ord'] = np.unique(df_enc.year, return_inverse=True)[1]

In [11]:
# Encode model categorical as ordinal 
model_decoder = np.unique(df_enc.model)
df_enc.model = np.unique(df_enc.model, return_inverse=True)[1]

# Encode make categorical as ordinal 
make_decoder = np.unique(df_enc.make)
df_enc.make = np.unique(df_enc.make, return_inverse=True)[1]

# Encode class categorical as ordinal 
class_decoder = np.unique(df_enc['class'])
df_enc['class'] = np.unique(df_enc['class'], return_inverse=True)[1]

In [12]:
# Dealing with NANS
df_enc.loc[df_enc.mileage.isnull(), 'mileage'] = 9999999999999

In [13]:
# Write Ouput file 
df_enc.to_csv('car_auction_enc.csv', index=False)